In [2]:
from overlaps import (
    read_overlaps_from_file,
)

In [34]:
overlaps = read_overlaps_from_file(OVERLAP_FILENAME)
false_pos, false_neg = get_misclassifications(overlaps)
print(len(false_pos), len(false_neg))

133 112


In [36]:
for idx in false_pos:
	print(idx)
	patterns=["TargetGene.*", "powerlaw.Score.*", "^start|end.*", "DistanceToTSS.*"]
	print(overlaps.iloc[idx].filter(regex="|".join(patterns)))
	print()

24
start_crispr                  12886091
end_crispr                    12887931
TargetGeneTSS_crispr          12881271
endTSS_crispr               12881468.0
TargetGene_crispr               DNASE2
DistanceToTSS_crispr             -4820
start_pred                    12886953
end_pred                      12887890
TargetGene_pred                 DNASE2
TargetGeneEnsemblID_pred           NaN
TargetGeneTSS_pred            12881271
powerlaw.Score_pred           0.094419
DistanceToTSS_pred               -5682
Name: 24, dtype: object

87
start_crispr                  12885011
end_crispr                    12885511
TargetGeneTSS_crispr          12881271
endTSS_crispr               12881468.0
TargetGene_crispr               DNASE2
DistanceToTSS_crispr             -3740
start_pred                    12884882
end_pred                      12885891
TargetGene_pred                 DNASE2
TargetGeneEnsemblID_pred           NaN
TargetGeneTSS_pred            12881271
powerlaw.Score_pred            0.